# 自动微分

假设我们想对函数$y=2\mathbf{x}^{\top}\mathbf{x}$关于列向量$\mathbf{x}$求导

In [14]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

在我们计算$y$关于$\mathbf{x}$的梯度之前，需要一个地方来存储梯度

In [15]:
x.requires_grad_(True) # 等价于x = torch.arange(4.0, requires_grad=True)
x.grad

现在计算$y$

$y=2\mathbf{\mathbf{x}}^{\top}\mathbf{\mathbf{x}}$

要求 $y$ 关于 $\mathbf{x}$ 的梯度，我们可以使用梯度下降或自动微分等方法。首先，我们将 $y$ 表示为 $\mathbf{x}$ 的函数，并求导。

给定 $\mathbf{x} = [0, 1, 2, 3]$ 和 $y = 2 \mathbf{x} ^\top \mathbf{x}$，其中 $\mathbf{x}^\top$ 表示 $\mathbf{x}$ 的转置。我们可以展开 $y$ 的表达式：

$y = 2 \mathbf{x} ^\top \mathbf{x} = 2(x_1^2 + x_2^2 + x_3^2 + x_4^2)$

我们可以将其表示为分量的形式：

$y = 2(x_1^2) + 2(x_2^2) + 2(x_3^2) + 2(x_4^2)$

现在，我们可以求每个分量关于 $\mathbf{x}$ 的偏导数：

$\frac{\partial y}{\partial x_1} = \frac{\partial}{\partial x_1} (2x_1^2) = 4x_1$

$\frac{\partial y}{\partial x_2} = \frac{\partial}{\partial x_2} (2x_2^2) = 4x_2$

$\frac{\partial y}{\partial x_3} = \frac{\partial}{\partial x_3} (2x_3^2) = 4x_3$

$\frac{\partial y}{\partial x_4} = \frac{\partial}{\partial x_4} (2x_4^2) = 4x_4$

因此，$y$ 关于 $\mathbf{x}$ 的梯度向量为：

$\nabla y = \left[\frac{\partial y}{\partial x_1}, \frac{\partial y}{\partial x_2}, \frac{\partial y}{\partial x_3}, \frac{\partial y}{\partial x_4}\right] = \left[4x_1, 4x_2, 4x_3, 4x_4\right] = 4 \mathbf{x}$

代入 $\mathbf{x} = [0, 1, 2, 3]$，我们可以计算梯度向量的值：

$\nabla y = \left[4(0), 4(1), 4(2), 4(3)\right] = [0, 4, 8, 12]$

因此，$y$ 关于 $\mathbf{x}$ 的梯度向量为 $[0, 4, 8, 12]$。

In [16]:
print(x)
print(x.grad)
print(torch.dot(x, x)) # x.T * x
y = 2 * torch.dot(x, x) # y是一个标量 内积 y = 2 * x.T * x
y

tensor([0., 1., 2., 3.], requires_grad=True)
None
tensor(14., grad_fn=<DotBackward0>)


tensor(28., grad_fn=<MulBackward0>)

通过调用反向传播函数来自动计算`y`关于`x`每个分量的梯度

In [17]:
y.backward()

In [18]:
x.grad == 4 * x

tensor([True, True, True, True])

现在计算`x`的另一个函数

In [19]:
x.grad.zero_()  # 重置梯度, 否则梯度会累加 下划线表示in-place操作
y = x.sum() # y是一个标量
y.backward()
x.grad

tensor([1., 1., 1., 1.])

深度学习中
，我们的目的不是计算微分矩阵，而是单独计算批量中每个样本的偏导数之和

给定 $y = x \cdot x$，我们可以将其表示为分量的形式：

$y = x_1 \cdot x_1 + x_2 \cdot x_2 + \ldots + x_n \cdot x_n$

现在，我们可以求每个分量关于 $x$ 的偏导数：

$\frac{\partial y}{\partial x_1} = \frac{\partial}{\partial x_1} (x_1 \cdot x_1) = 2x_1$

$\frac{\partial y}{\partial x_2} = \frac{\partial}{\partial x_2} (x_2 \cdot x_2) = 2x_2$

$\ldots$

$\frac{\partial y}{\partial x_n} = \frac{\partial}{\partial x_n} (x_n \cdot x_n) = 2x_n$

因此，$y$ 关于 $x$ 的梯度向量为：

$\nabla y = \left[\frac{\partial y}{\partial x_1}, \frac{\partial y}{\partial x_2}, \ldots, \frac{\partial y}{\partial x_n}\right] = \left[2x_1, 2x_2, \ldots, 2x_n\right]$

所以，$y$ 关于 $x$ 的梯度向量为 $[2x_1, 2x_2, \ldots, 2x_n]$。

In [20]:
x.grad.zero_()
y = x * x
print(y)
y.sum().backward()
x.grad

tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>)


tensor([0., 2., 4., 6.])

将某些计算移动到记录的计算图之外

In [8]:
x.grad.zero_()
y = x * x
u = y.detach() # 相当于把y看成常数
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [9]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），我们仍然可以计算得到的变量的梯度

In [22]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

# 隐式

a = torch.randn(size=(), requires_grad=True)
print(a)
d = f(a)
print(d)
d.backward()

a.grad == d / a

tensor(-2.3988, requires_grad=True)
tensor(-122820.3750, grad_fn=<MulBackward0>)


tensor(True)